In [32]:
import requests
from bs4 import BeautifulSoup
import datetime as dt
import time
from datetime import datetime
from decouple import config
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

In [33]:
def scrape_links_news(yesterday,date, keywords):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://jateng.antaranews.com/search?q={keywords}&startDate={yesterday}&endDate={date}&submit=Submit"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article',{"class": "simple-post simple-big clearfix"})

    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    print(f"data{len(page_links)} and url {url}")
    # logger.success({
    #     "message": f"Scraped {len(page_links)} links from page {page_number} when {date}"
    #     })
    
    return page_links

In [34]:
def scrape_link_per_day(date, keywords,max_threads=5):
    page_links = None
    date = dt.datetime.strptime(date, "%Y-%m-%d")
    formatted_date = date.strftime("%d-%m-%Y")
    
    yesterday = date - dt.timedelta(days=1)
    yesterday = yesterday.strftime("%d-%m-%Y")
    
    page_links= scrape_links_news(yesterday,formatted_date, keywords)


    return page_links

In [35]:
link=scrape_link_per_day("2023-11-01",'jokowi')

data1 and url https://jateng.antaranews.com/search?q=jokowi&startDate=31-10-2023&endDate=01-11-2023&submit=Submit


In [36]:
print(link)

['https://jateng.antaranews.com/berita/513321/ikn-access-toll-road-work-55-percent-complete-jokowi']


In [37]:
def scrape_url(url,max_retries=2):
    retries = 0
    while retries < max_retries:
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
                }
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')

                    # element Title 
                    title_elem = soup.find('h1',{"class": "post-title"})
                    title_text = title_elem.text.strip() if title_elem else "Title not found"
                    # element Date
                    date_elem = soup.find('span', {"class": "article-date"})
                    date_text = date_elem.text.strip() if date_elem else "Date not found"
                    # locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')
                    month_names_id = {
                        'Januari': 'January',
                        'Februari': 'February',
                        'Maret': 'March',
                        'April': 'April',
                        'Mei': 'May',
                        'Juni': 'June',
                        'Juli': 'July',
                        'Agustus': 'August',
                        'September': 'September',
                        'Oktober': 'October',
                        'November': 'November',
                        'Desember': 'December',
                    }

                    day_names_id = {
                        'Senin': 'Monday',
                        'Selasa': 'Tuesday',
                        'Rabu': 'Wednesday',
                        'Kamis': 'Thursday',
                        'Jumat': 'Friday',
                        'Sabtu': 'Saturday',
                        'Minggu': 'Sunday',
                    }

                    date_text = "Thursday, 23 November 2023 14:07 WIB"

                    for ind, eng in month_names_id.items():
                        date_text = date_text.replace(ind, eng)

                    for ind, eng in day_names_id.items():
                        date_text = date_text.replace(ind, eng)

                    date_obj = datetime.strptime(date_text, '%A, %d %B %Y %H:%M %Z')
                    formatted_date = date_obj.strftime('%Y-%m-%d %H:%M:%S')

                    body_elem = soup.find('div', {"itemprop": "articleBody"})
                    if body_elem:
                        content_text = body_elem.text
                        content_text = content_text.replace('\n', '').replace('\r', '').replace('\t', '')
                        content_text = ' '.join(content_text.split())
                    else:
                        content_text ="Content not found"

                    return {
                        'title': title_text,
                        'date': formatted_date,
                        'content': content_text,
                        'link': url
                    }
                else:
                    # logger.error(
                    #     {"message": f"Failed to retrieve data from {url}: Status Code {response.status_code}"}
                    #     )
                    print('hellow')
            except requests.exceptions.RequestException as e:
                # logger.error({
                #     "message":f"Error fetching URL '{url}': {e}"
                # })
                print(e)
            except Exception as e:
                # logger.error({
                #     "message":f"Error processing URL '{url}': {e}"
                # })
                print(e)
            retries += 1
            if retries < max_retries:
                # logger.info({
                #     "message":f"Retrying {url} (Attempt {retries}/{max_retries})"
                # })
                print(f"Retrying {url} (Attempt {retries}/{max_retries})")
                time.sleep(5)  # You can adjust the delay as needed
    return None       


In [38]:
url='https://jateng.antaranews.com/berita/515802/jelang-akhir-tugas-dedy-yon-pamitan-ke-dprd-kota-tegal'
cek=scrape_url(url)

In [39]:
print(cek)

{'title': 'Jelang akhir tugas, Dedy Yon pamitan ke DPRD Kota Tegal', 'date': '2023-11-23 14:07:00', 'content': 'Tegal (ANTARA) - Menjelang berakhirnya masa jabatan Wali Kota Tegal, H. Dedy Yon Supriyono yang masa tugasnya berakhir pada 31 Desember 2023, pamitan kepada anggota DPRD Kota Tegal. Pamitan tersebut disampaikan Dedy Yon pada Rapat Paripurna DPRD Kota Tegal dengan agenda Pengumuman Pemberhentian Wali Kota Tegal dan Wakil Wali Kota Tegal Masa Jabatan 2019-2023, Kamis (23/11/2023).Wali Kota Tegal Dedy Yon dan Wakil Wali Kota Jumadi menunaikan tugas pada periode 2019-2024.Dalam kesempatan tersebut Wali Kota Tegal Dedy Yon Supriyono menyampaikan bahwa semua perjalanan memiliki batas akhir, setiap pertemuan ada perpisahan.“Batas waktu saya untuk melaksanakan pengabdian sebagai Wali Kota Tegal akan berakhir 31 Desember 2023 nanti,” ujar Wali Kota Tegal. Ia menyampaikan ucapan terima kasih kepada masyarakat Kota Tegal dan wakil rakyat yang telah memberi kepercayaan kepadanya untuk me